In [1]:
# scoreing 파일

In [1]:
import os
import math
from tqdm import tqdm_notebook as tqdm
import time
import numpy as np
import pandas as pd
import logging
import argparse

import warnings
import multiprocessing
warnings.filterwarnings("ignore")

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from deepctr.models import DeepFM
from deepctr.feature_column import SparseFeat,DenseFeat, get_feature_names, VarLenSparseFeat

In [2]:
import os
from os.path import isfile, join
basedir = '/home/lms/ms/poc'
rawdata_path = join(basedir, 'dataset')
procdata_path = join(basedir, 'deepfm', 'dataset', 'preprocess', 'rating')
result_path_5_no_f = join(basedir, 'deepfm', 'result', 'result_5_no_f') # path 설정 필요
result_path_r_no_f = join(basedir, 'deepfm', 'result', 'result_r_no_f') # path 설정 필요
result_path_5_f = join(basedir, 'deepfm', 'result', 'result_5_f') # path 설정 필요
result_path_r_f = join(basedir, 'deepfm', 'result', 'result_r_f') # path 설정 필요

model_path_5_no_f= join(basedir, 'deepfm', 'model', 'model_5_no_f')
model_path_r_no_f= join(basedir, 'deepfm', 'model', 'model_r_no_f')
model_path_5_f= join(basedir, 'deepfm', 'model', 'model_5_f')
model_path_r_f= join(basedir, 'deepfm', 'model', 'model_r_f')

batch_size = 200
topk = 25

In [3]:
# 추천리스트 load
df0 = pd.read_csv(os.path.join(result_path_5_f, 'reclist_0.csv'), sep='\001', header=None)
df0.head()

,0,1,2,3,4,5
0,VARIETY,0,94,런닝맨,4.89754,1
1,VARIETY,0,12,놀면 뭐하니?,4.89375,2
2,VARIETY,0,59,나 혼자 산다,4.87988,3
3,VARIETY,0,0,꼬리에 꼬리를 무는 그날 이야기 시즌2,4.87470,4
4,VARIETY,0,16,꼬리에 꼬리를 무는 그날 이야기,4.86291,5


In [4]:
#load test data
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
data = pd.read_pickle(os.path.join(procdata_path, 'hist.pkl'))
train, test = train_test_split(data, test_size=0.2, random_state=2020)
test.head()

,sa_id_CODE,category_id_CODE,rating_r,rating_5,seg_1,seg_2,seg_3,seg_4,pr_info_CODE,price_CODE,release_date,run_time_ss
10838611,503022,1188,4.50000,5,0.85,0.0,0.07,0.08,0,2,0.926502,0.201008
15979174,652914,11948,4.50000,5,0.87,0.0,0.00,0.13,0,2,0.947011,0.192812
4541125,502835,166,5.00000,5,0.98,0.0,0.01,0.01,3,0,0.999991,0.155527
14053222,352951,3104,4.50000,5,0.94,0.0,0.00,0.06,0,3,0.989897,0.170110
8806112,643387,669,2.61375,5,1.00,0.0,0.00,0.00,1,3,0.883084,0.240833


In [5]:
test_set = test.sort_values(by=['sa_id_CODE', 'category_id_CODE'], ascending=[True, True], axis=0)
test_set.head()

,sa_id_CODE,category_id_CODE,rating_r,rating_5,seg_1,seg_2,seg_3,seg_4,pr_info_CODE,price_CODE,release_date,run_time_ss
203378,1,1,5.00000,5,0.97,0.0,0.03,0.00,0,0,0.990313,0.131286
318085,1,4,5.00000,5,0.97,0.0,0.03,0.00,2,0,0.979802,0.163261
548474,2,12,4.50000,5,0.97,0.0,0.03,0.00,0,0,0.999996,0.186040
1873142,3,44,4.57825,5,0.98,0.0,0.01,0.01,2,1,0.989588,0.104236
1921621,3,50,5.00000,5,0.98,0.0,0.01,0.01,0,0,0.841609,0.137712


In [6]:
# test set에 있는 유저 추출
user_set = test_set['sa_id_CODE'].unique()
user_set = list(user_set)
user_set = [ int(x) for x in user_set]

In [7]:
# test set에 있는 유저에 대한 추천리스트 추출
pred = df0[df0[1].isin(user_set)]
pred.head()

,0,1,2,3,4,5
25,VARIETY,1,94,런닝맨,4.94071,1
26,VARIETY,1,0,꼬리에 꼬리를 무는 그날 이야기 시즌2,4.91583,2
27,VARIETY,1,12,놀면 뭐하니?,4.91408,3
28,VARIETY,1,59,나 혼자 산다,4.91221,4
29,VARIETY,1,29,스트릿 우먼 파이터,4.88538,5


In [ ]:
# 참고.....
df1 = pd.read_csv(os.path.join(model_pred_path, date, 'reclist_1.csv'), sep='\001', header=None)
df2 = pd.read_csv(os.path.join(model_pred_path, date, 'reclist_2.csv'), sep='\001', header=None)
# test set2 에 있는 유저에 대한 추천리스트 추출
pred_set2 = df1[df1[1].isin(user_set)]
pred_set3 = df2[df2[1].isin(user_set)]
pred1_set2 = pred_set2.groupby(1)[2].apply(list)
pred1_set3 = pred_set3.groupby(1)[2].apply(list)
predict_set2 = []
for i in pred1_set2:
    predict_set2.append(i)
predict_set3 = []
for i in pred1_set3:
    predict_set3.append(i)

In [8]:
pred1 = pred.groupby(1)[2].apply(list)

In [9]:
# 추천리스트를 리스트로
predict = []
for i in pred1:
    predict.append(i)

In [10]:
# test set을 리스트로
la = test_set.groupby('sa_id_CODE')['category_id_CODE'].apply(list)
label = []
for i in la:
    label.append(i)

In [11]:
# 전체 testset 중 일부만 (현재는 약 절반) 대상
target = label[:len(predict)]
len(target) #####

617569

### 지표 시작!!!

In [12]:
def calc_precision(predicted, actual):
        prec = [value for value in predicted if value in actual]
        prec = np.round(float(len(prec)) / float(len(predicted)), 4)
        return prec

In [13]:
pred_at = np.mean(list(map(calc_precision, predict, target)))

In [14]:
pred_at ################## precision

0.03426473802927283

In [15]:
def calc_recall(predicted, actual):
    reca = [value for value in predicted if value in actual]
    reca = np.round(float(len(reca)) / float(len(actual)), 4)
    return reca

In [16]:
recall_at = np.mean(list(map(calc_recall, predict, target)))

In [17]:
recall_at  ###################### recall

0.3075456191939686

In [18]:
def prediction_coverage(predicted, tot):
    predicted_flattened = [p for sublist in predicted for p in sublist]
    unique_predictions = len(set(predicted_flattened))
    prediction_coverage = round(unique_predictions/tot*100,2)
    return prediction_coverage

In [19]:
tot = data['category_id_CODE'].nunique()

In [20]:
cov_at = prediction_coverage(predict,tot)

In [21]:
print(cov_at, "%")  ############### coverage

8.57 %


In [22]:
def mrr(prediction, label):
    rr = 0
    for i in range(len(prediction)):
        if any(np.isin(prediction[i], label[i]))==False:
            continue
        else:
            rank = np.min(np.where(np.isin(prediction[i], label[i])))+1
        rr = rr + 1/rank
    return rr/len(prediction)

In [23]:
mrr_at = mrr(predict, target)
mrr_at ########################### mrr

0.15233382110843843

### negative sampling 테스트 (p@25 / r@25 / c@25 / m@25)
#### rating_5_no_f : 0.032 / 0.29 / 0.09 / 0.15
#### rating_r_no_f : 0.030 / 0.27 / 0.13 / 0.14 
#### rating_5_f : 0.034 / 0.31 / 0.09 / 0.15
#### rating_r_f : 0.031 / 0.29 / 0.09 / 0.14

# 기존 상용 지표
### 평점 : (상용 모델) 시청시간 + 최신성 => 0.0035 / 0.027 / 0.46% / 0.012
### 평점 : 0.5 + 시청시간 + 최신성 => 0.005 / 0.04 / 0.6% / 0.015 (1/24)
### 평점 : 5 (epoch 5) => 0.0001 / 0.0004 / 1.11% / 0.0001 (1/25)
### 평점 : 5 (epoch 10) => 안좋음... (1/25_1)
### 평점 : 0.5 + 시청 시간 + 최신성 + 네버티브 샘플링(0.1, 5개) => 0.0078 / 0.068 / 0.71% / 0.0198

### 2번째평가 함수들

In [75]:
def precision_at(predictions, labels, k=10, assume_unique=True):
    # validate K
    # _require_positive_k(k)

    def _inner_pk(pred, lab):
        # need to compute the count of the number of values in the predictions
        # that are present in the labels. We'll use numpy in1d for this (set
        # intersection in O(1))
        if lab.shape[0] > 0:
            n = min(pred.shape[0], k)
            cnt = np.in1d(pred[:n], lab, assume_unique=assume_unique).sum()
            return float(cnt) / k
        else:
            return _warn_for_empty_labels()

    return _mean_ranking_metric(predictions, labels, _inner_pk)

In [76]:
def _mean_ranking_metric(predictions, labels, metric):
    """Helper function for precision_at_k and mean_average_precision"""
    # do not zip, as this will require an extra pass of O(N). Just assert
    # equal length and index (compute in ONE pass of O(N)).
    # if len(predictions) != len(labels):
    #     raise ValueError("dim mismatch in predictions and labels!")
    # return np.mean([
    #     metric(np.asarray(predictions[i]), np.asarray(labels[i]))
    #     for i in xrange(len(predictions))
    # ])
    
    # Actually probably want lazy evaluation in case preds is a 
    # generator, since preds can be very dense and could blow up 
    # memory... but how to assert lengths equal? FIXME
    return np.mean([
        metric(np.asarray(prd), np.asarray(labels[i]))
        for i, prd in enumerate(predictions)  # lazy eval if generator
    ])

In [79]:
def mean_average_precision(predictions, labels, assume_unique=True):
    def _inner_map(pred, lab):
        if lab.shape[0]:
            # compute the number of elements within the predictions that are
            # present in the actual labels, and get the cumulative sum weighted
            # by the index of the ranking
            n = pred.shape[0]

            # Scala code from Spark source:
            # var i = 0
            # var cnt = 0
            # var precSum = 0.0
            # val n = pred.length
            # while (i < n) {
            #     if (labSet.contains(pred(i))) {
            #         cnt += 1
            #         precSum += cnt.toDouble / (i + 1)
            #     }
            #     i += 1
            # }
            # precSum / labSet.size

            arange = np.arange(n, dtype=np.float32) + 1.  # this is the denom
            present = np.in1d(pred[:n], lab, assume_unique=assume_unique)
            prec_sum = np.ones(present.sum()).cumsum()
            denom = arange[present]
            return (prec_sum / denom).sum() / lab.shape[0]

        else:
            return _warn_for_empty_labels()

    return _mean_ranking_metric(predictions, labels, _inner_map)


In [ ]:
def mrr_score(y_true, y_score):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order)
    rr_score = y_true / (np.arange(len(y_true)) + 1)
    return np.sum(rr_score) / np.sum(y_true)

In [301]:
pp = [[1,2,3,4,5], [3,4,5,6,7], [1,2,3,4,5]]
aa = [[6], [2,3,7], [4,1]]

In [217]:
rr=0
for i in range(len(pp)):
    rank = np.min(np.where(np.isin(pp[i], aa[i])))+1
    rr = rr + 1/rank

In [305]:
def mrr(prediction, label):
    rr = 0
    for i in range(len(prediction)):
        print(i)
        if any(np.isin(prediction[i], label[i]))==False:
            continue
        else:
            rank = np.min(np.where(np.isin(prediction[i], label[i])))+1
        rr = rr + 1/rank
        print(rank)
    return rr/len(prediction)

In [ ]:
# MAP
map_k = mean_average_precision(predict, target)
map_k

In [ ]:
### 방법2

## 추론 테스트

In [ ]:
df0 = pd.read_csv(os.path.join(model_pred_path, date, 'reclist_0.csv'), sep='\001', header=None)
df0.head()

In [86]:
df1 = pd.read_csv(os.path.join(model_pred_path, date, 'reclist_1.csv'), sep='\001', header=None)
df1.head()

,0,1,2,3,4,5
0,DRAMA,889800,922,두 번째 남편,0.95816,1
1,DRAMA,889800,286,사랑의 꽈배기,0.95523,2
2,DRAMA,889800,426,고스트 닥터,0.95084,3
3,VARIETY,889800,1060,인간극장 2022,0.94818,4
4,DRAMA,889800,461,고스트 닥터,0.94186,5


In [87]:
df2 = pd.read_csv(os.path.join(model_pred_path, date, 'reclist_2.csv'), sep='\001', header=None)
df2.head()

,0,1,2,3,4,5
0,DRAMA,1762400,922,두 번째 남편,0.94968,1
1,DRAMA,1762400,286,사랑의 꽈배기,0.93126,2
2,DRAMA,1762400,426,고스트 닥터,0.92703,3
3,DRAMA,1762400,143,국가대표 와이프,0.92345,4
4,VARIETY,1762400,1233,한국기행(2022),0.92108,5


In [24]:
df0[df0[5]==1].head(30)

,0,1,2,3,4,5
0,VARIETY,0,94,런닝맨,4.63482,1
25,VARIETY,1,94,런닝맨,4.73822,1
50,VARIETY,2,94,런닝맨,4.70342,1
75,VARIETY,3,94,런닝맨,4.74211,1
100,VARIETY,4,94,런닝맨,4.53118,1
125,DRAMA,5,19,두 번째 남편,4.83208,1
150,VARIETY,6,94,런닝맨,4.78852,1
175,VARIETY,7,94,런닝맨,4.71197,1
200,VARIETY,8,94,런닝맨,4.69788,1
225,VARIETY,9,94,런닝맨,4.63511,1


In [24]:
df0[df0[1]==63]

,0,1,2,3,4,5
1575,VARIETY,63,94,런닝맨,5.03557,1
1576,VARIETY,63,12,놀면 뭐하니?,5.02019,2
1577,VARIETY,63,59,나 혼자 산다,5.01207,3
1578,VARIETY,63,0,꼬리에 꼬리를 무는 그날 이야기 시즌2,5.01173,4
1579,VARIETY,63,16,꼬리에 꼬리를 무는 그날 이야기,4.97634,5
1580,MOVIE,63,100,싱크홀(2021),4.97561,6
1581,VARIETY,63,29,스트릿 우먼 파이터,4.97395,7
1582,VARIETY,63,32,그것이 알고 싶다,4.96780,8
1583,VARIETY,63,20,심야괴담회,4.96454,9
1584,VARIETY,63,129,미운 우리 새끼,4.96227,10


In [ ]:
926380 / 1139744 / 370071 / 775896 / 825254 / 1246938

In [27]:
mapping = pd.read_pickle(os.path.join(procdata_path, 'mapping.pkl'))
mapping

,sa_id,category_id,category_id_CODE,sa_id_CODE
0,205090929633,B3054,0,0
1,206059946955,B3054,0,1
2,207044266005,B3054,0,2
3,207044353341,B3054,0,3
4,208036007235,B3054,0,4
...,...,...,...,...
16044099,500228395772,M012141362PPV00,17216,1973456
16044100,500233024250,M012057404PPV00,17217,1973457
16044101,500079172794,M01208Q414PPV00,17218,1973458
16044102,500131191860,M01163M037PPV00,17219,1973459


In [29]:
mapping[mapping['sa_id']=='500009388201']

,sa_id,category_id,category_id_CODE,sa_id_CODE
63,500009388201,B3054,0,63
318128,500009388201,B3042,4,63
1009361,500009388201,B50O3,20,63
2351635,500009388201,M012179351PPV00,92,63
2956041,500009388201,M0121A7389PPV00,99,63
3479561,500009388201,M01218Q018PPV00,106,63
3751162,500009388201,M0121BI140PPV00,114,63
3900587,500009388201,M01197T066PPV00,123,63
4020911,500009388201,B567O,129,63
4165225,500009388201,B41TL,131,63


In [28]:
mapping[mapping['sa_id_CODE']==63]

,sa_id,category_id,category_id_CODE,sa_id_CODE
63,500009388201,B3054,0,63
318128,500009388201,B3042,4,63
1009361,500009388201,B50O3,20,63
2351635,500009388201,M012179351PPV00,92,63
2956041,500009388201,M0121A7389PPV00,99,63
3479561,500009388201,M01218Q018PPV00,106,63
3751162,500009388201,M0121BI140PPV00,114,63
3900587,500009388201,M01197T066PPV00,123,63
4020911,500009388201,B567O,129,63
4165225,500009388201,B41TL,131,63


In [30]:
items = pd.read_pickle(os.path.join(procdata_path, 'item.pkl'))
item_columns = ['category_id','gubun','rep_album_id','category_name']
category_data = pd.read_pickle(os.path.join(rawdata_path, 'item.pkl'))
category_data = category_data.astype({'category_id':'str', 'category_name':'str', 'album_id':'str'})
category_data = category_data.drop_duplicates(subset=item_columns)
category_data = items.merge(category_data, on='category_id')

In [31]:
map_category = category_data[['category_name', 'category_id_CODE', 'dpeth0_nm']]
map_category

,category_name,category_id_CODE,dpeth0_nm
0,꼬리에 꼬리를 무는 그날 이야기 시즌2,0,예능시사교양
1,표리부동,1,예능시사교양
2,쇼윈도：여왕의 집,2,드라마
3,워맨스가 필요해,3,예능시사교양
4,꼬리에 꼬리를 무는 그날 이야기 시즌1,4,예능시사교양
...,...,...,...
17216,에비에이터(소장),17216,영화
17217,딥워터 호라이즌,17217,영화
17218,위 캔 두 댓! 가치봄영화,17218,영화
17219,텍사스 레인저,17219,영화


In [32]:
mapping1 = pd.merge(mapping, map_category, how="left", on="category_id_CODE")
mapping1

,sa_id,category_id,category_id_CODE,sa_id_CODE,category_name,dpeth0_nm
0,205090929633,B3054,0,0,꼬리에 꼬리를 무는 그날 이야기 시즌2,예능시사교양
1,206059946955,B3054,0,1,꼬리에 꼬리를 무는 그날 이야기 시즌2,예능시사교양
2,207044266005,B3054,0,2,꼬리에 꼬리를 무는 그날 이야기 시즌2,예능시사교양
3,207044353341,B3054,0,3,꼬리에 꼬리를 무는 그날 이야기 시즌2,예능시사교양
4,208036007235,B3054,0,4,꼬리에 꼬리를 무는 그날 이야기 시즌2,예능시사교양
...,...,...,...,...,...,...
16044099,500228395772,M012141362PPV00,17216,1973456,에비에이터(소장),영화
16044100,500233024250,M012057404PPV00,17217,1973457,딥워터 호라이즌,영화
16044101,500079172794,M01208Q414PPV00,17218,1973458,위 캔 두 댓! 가치봄영화,영화
16044102,500131191860,M01163M037PPV00,17219,1973459,텍사스 레인저,영화


In [33]:
mapping1[mapping1['sa_id_CODE']==63]

,sa_id,category_id,category_id_CODE,sa_id_CODE,category_name,dpeth0_nm
63,500009388201,B3054,0,63,꼬리에 꼬리를 무는 그날 이야기 시즌2,예능시사교양
318128,500009388201,B3042,4,63,꼬리에 꼬리를 무는 그날 이야기 시즌1,예능시사교양
1009361,500009388201,B50O3,20,63,심야괴담회,예능시사교양
2351635,500009388201,M012179351PPV00,92,63,발신제한,영화
2956041,500009388201,M0121A7389PPV00,99,63,모가디슈,영화
3479561,500009388201,M01218Q018PPV00,106,63,올드(2021),영화
3751162,500009388201,M0121BI140PPV00,114,63,강릉,영화
3900587,500009388201,M01197T066PPV00,123,63,어벤져스: 엔드게임(소장),영화
4020911,500009388201,B567O,129,63,미운 우리 새끼,예능시사교양
4165225,500009388201,B41TL,131,63,벌거벗은 세계사,예능시사교양


In [ ]:
################# 평점 보기

In [28]:
data = pd.read_pickle(os.path.join(procdata_path, 'hist.pkl'))
data.head()

,sa_id_CODE,category_id_CODE,rating,seg_1,seg_2,seg_3,seg_4,pr_info_CODE,price_CODE,release_date,run_time_ss
0,0,0,0.4639,1.00,0.0,0.00,0.00,0,0,0.990018,0.17365
1,1,0,0.5611,0.97,0.0,0.03,0.00,0,0,0.990018,0.17365
2,2,0,0.4361,0.97,0.0,0.03,0.00,0,0,0.990018,0.17365
3,3,0,0.9694,0.98,0.0,0.01,0.01,0,0,0.990018,0.17365
4,4,0,0.4417,0.86,0.0,0.01,0.13,0,0,0.990018,0.17365


In [31]:
data[data['sa_id_CODE']==370071]

,sa_id_CODE,category_id_CODE,rating1,seg_1,seg_2,seg_3,seg_4,pr_info_CODE,price_CODE,release_date,run_time_ss
612081,370071,12,0.85727,0.9,0.0,0.1,0.0,0,0,0.999996,0.186040
2532734,370071,94,0.84222,0.9,0.0,0.1,0.0,0,0,0.999997,0.234407
